# Battle of Neighborhoods in Manhattan

<hr>

### About Manhattan

> Manhattan is a major central city for diversity since many people from different cultural atmospheres have brought their families and dreams to Manhattan. 

>The city has consistently seen people from around the world move to the city and call it home. It has been a center for trade and economic growth. 

>Manhattan is known world wide as a cultural melting pot. While other states have had immigration surges, none have compared to the diversity and sheer number of immigrants that have made their way to the City. 

>So these number of varies cultures combined to create a great diversity for itself. Since People from all over the world tend to come up here, we can see some many of their cultural aspects Transport, Food, Clothing, and so on…

### Problem Description

>Restaurant is a place where people come to have food and drinks for a cost, People love to do many things and try something new or stick with their own routines, it depends on the individual and there are so many of them with different cultural and various aspects in Manhattan. 

>There are so many cuisines, which is based on the style of cooking, the ingredients, dishes and techniques. For our problem lets stick with Indian cuisine.

>Let’s assume in this one of the world’s most diverse region we want to open an Indian restaurant, so what are all the factors we have to take into account such as follows,
* Market Places
* Competition in particular location
* Aiding places that make people come to restaurants like Gym,  * Entertaining Public places
* Population
* Menu from competitors

>And so on… So our solution needs to be data driven for avoiding or considering low risk criteria and high success rate and thus apply our overall knowledge in the techniques and the tools gained so far in this course.
<hr>

> Here addresses is converted into their equivalent latitude and longitude values and also used `Foursquare API` to explore neighborhoods in Manhattan.

<hr>

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from pandas.io.json import json_normalize

import json
import requests

from matplotlib import cm, colors

import folium
from geopy.geocoders import Nominatim

#### Getting co-ordinates of locations

In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

# newyork_data

All the relevant data is in the `features key`, which is basically a list of the neighborhoods

In [3]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Transforming the data to pandas dataframe

In [4]:
columns = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

df = pd.DataFrame(columns=columns)
df

,Borough,Neighborhood,Latitude,Longitude


In [5]:
# Populating the dataframe
for data in neighborhoods_data:
    borough = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    
    neighborhood_coordinates = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_coordinates[1]
    neighborhood_long = neighborhood_coordinates[0]
    
    df = df.append({
        'Borough': borough,
        'Neighborhood': neighborhood_name,
        'Latitude': neighborhood_lat,
        'Longitude': neighborhood_long
    }, ignore_index=True)

In [6]:
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [7]:
unique_borough = len(df['Borough'].unique())
num_of_neighborhood = df.shape[0]

print(f'The dataframe has {unique_borough} boroughs and {num_of_neighborhood} neighborhoods.')

The dataframe has 5 boroughs and 306 neighborhoods.


#### Using geopy library to get the latitude and longitude values of New York City

In [8]:
address = 'New York City, NY'

geolocater = Nominatim(user_agent='ny_explorer')
location = geolocater.geocode(address)
latitude = location.latitude
longitude =location.longitude

print(f'Geographical Coordinates of New York City are {latitude}, {longitude}')

Geographical Coordinates of New York City are 40.7127281, -74.0060152


#### Creating a map of New York with neighborhoods superimposed on top

In [9]:
map_ny = folium.Map(location=[latitude, longitude], zoom_start=10)

# Adding markers on top
for lat, lng, borough, neigh in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = f'{neigh}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False
    ).add_to(map_ny)
    
map_ny

In [10]:
df.Borough.unique().tolist()

['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island']

<hr>

## Going for Manhattan

In [11]:
mnh_df = df[df.Borough == 'Manhattan'].reset_index(drop=True)
mnh_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


#### Getting co-ordinates of Manhattan

In [12]:
address = 'Manhattan, NY'

geolocater = Nominatim(user_agent='manh_explorer')
location = geolocater.geocode(address)
latitude = location.latitude
longitude =location.longitude

print(f'Geographical Coordinates of Manhattan are {latitude}, {longitude}')

Geographical Coordinates of Manhattan are 40.7896239, -73.9598939


#### Visualizing Manhattan

In [13]:
map_mnh = folium.Map(location=[latitude, longitude], zoom_start=10)

# Adding markers on top
for lat, lng, borough, neigh in zip(mnh_df['Latitude'], mnh_df['Longitude'], mnh_df['Borough'], mnh_df['Neighborhood']):
    label = f'{neigh}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False
    ).add_to(map_mnh)
    
map_mnh

#### Defining Foursquare Credentials and Version

In [14]:
CLIENT_ID =  'your Foursquare ID'
CLIENT_SECRET = 'your Foursquare Secret'
VERSION = '20180605' # Foursquare API version

<hr>

## Exploring one neighborhood in Manhattan

In [15]:
# Neighborhood name
mnh_df.loc[0, 'Neighborhood']

'Marble Hill'

In [16]:
neigh_lat = mnh_df.loc[0, 'Latitude']
neigh_lng = mnh_df.loc[0, 'Longitude']
neigh_name = mnh_df.loc[0, 'Neighborhood']
print(f'Latitude and Longitude of {neigh_name} are {neigh_lat}, {neigh_lng}')

Latitude and Longitude of Marble Hill are 40.87655077879964, -73.91065965862981


####  Getting the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [17]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_lng, 
    radius,
    LIMIT
)

#### Making GET request to Foursquare API

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eba83c66001fe001b0ae82b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 26,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

**`The required information is in items key`**

#### Function to extract the category of the venue

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Cleaning the json and structuring it into a pandas dataframe

In [20]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-20-d27113591c25>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

26 venues were returned by Foursquare.


<hr>

### Exploring all the Neighborhood in Manhattan

#### Function to get venues near a location

In [22]:
def get_nearby_venues(names, latitudes, longitudes, radius=500):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
        )

        # Make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']

        # Return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']
        ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Neighborhood',
        'Neighborhood.Latitude',
        'Neighborhood.Longitude',
        'Venue',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category'
    ]
    
    return (nearby_venues)

#### Getting nearby neighborhood

In [23]:
# Running get_nearby_venues function on each neighborhood in Manhattan

mnh_venues = get_nearby_venues(
                    names=mnh_df.Neighborhood, 
                    latitudes=mnh_df.Latitude,
                    longitudes=mnh_df.Longitude
                )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [24]:
print(mnh_venues.shape)
mnh_venues.head()

(2996, 7)


,Neighborhood,Neighborhood.Latitude,Neighborhood.Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


#### Grouping venues on the basis of `Neighborhood`

In [25]:
mnh_venues.groupby('Neighborhood').count()

,Neighborhood.Latitude,Neighborhood.Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,59,59,59,59,59,59
Carnegie Hill,85,85,85,85,85,85
Central Harlem,45,45,45,45,45,45
Chelsea,97,97,97,97,97,97
Chinatown,100,100,100,100,100,100
Civic Center,87,87,87,87,87,87
Clinton,100,100,100,100,100,100
East Harlem,43,43,43,43,43,43
East Village,100,100,100,100,100,100


In [26]:
print('There are {} uniques categories.'.format(len(mnh_venues['Venue Category'].unique())))

There are 329 uniques categories.


#### Analyze each Neighborhood

In [27]:
# One Hot Encoding
mnh_oh = pd.get_dummies(mnh_venues[['Venue Category']], prefix='', prefix_sep='')

# add neighborhood column back to dataframe
mnh_oh['Neighborhood'] = mnh_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [mnh_oh.columns[-1]] + list(mnh_oh.columns[:-1])
mnh_oh = mnh_oh[fixed_columns]

mnh_oh.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soup Pla

In [28]:
mnh_oh.shape

(2996, 330)

**`Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category`**

In [29]:
mnh_group = mnh_oh.groupby('Neighborhood').mean().reset_index()
mnh_group

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soup Pla

In [30]:
mnh_group.shape

(40, 330)

#### Printing top 5 most common venues

In [31]:
num_top_venues = 5

for place in mnh_group.Neighborhood:
    print(f'==== {place} ====')
    tmp = mnh_group[mnh_group.Neighborhood == place].T.reset_index()
    tmp.columns = ['venue', 'freq']
    tmp = tmp.iloc[1:]
    tmp['freq'] = tmp['freq'].astype(float)
    tmp = tmp.round({'freq': 2})
    print(tmp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print()

==== Battery Park City ====
           venue  freq
0           Park  0.10
1          Hotel  0.07
2  Memorial Site  0.05
3  Boat or Ferry  0.05
4            Gym  0.05

==== Carnegie Hill ====
           venue  freq
0    Coffee Shop  0.08
1    Yoga Studio  0.04
2      Bookstore  0.04
3    Pizza Place  0.04
4  Grocery Store  0.04

==== Central Harlem ====
                 venue  freq
0   African Restaurant  0.07
1                  Bar  0.04
2  American Restaurant  0.04
3   Seafood Restaurant  0.04
4   Chinese Restaurant  0.04

==== Chelsea ====
                venue  freq
0         Art Gallery  0.16
1         Coffee Shop  0.06
2      Ice Cream Shop  0.03
3  Italian Restaurant  0.03
4           Juice Bar  0.02

==== Chinatown ====
                 venue  freq
0   Chinese Restaurant  0.09
1               Bakery  0.04
2         Cocktail Bar  0.04
3                  Spa  0.03
4  American Restaurant  0.03

==== Civic Center ====
               venue  freq
0        Coffee Shop  0.06
1          

#### Function to get most common venues

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

#### Getting top 10 most common venues in a Neighborhood

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neigh_venues_sorted = pd.DataFrame(columns=columns)
neigh_venues_sorted['Neighborhood'] = mnh_group['Neighborhood']

for ind in np.arange(mnh_group.shape[0]):
    neigh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mnh_group.iloc[ind, :], num_top_venues)

neigh_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Boat or Ferry,Memorial Site,Shopping Mall,Food Court,Beer Garden,Playground,Plaza
1,Carnegie Hill,Coffee Shop,Yoga Studio,Wine Shop,Pizza Place,Japanese Restaurant,Gym / Fitness Center,Gym,Grocery Store,Café,Bookstore
2,Central Harlem,African Restaurant,Chinese Restaurant,French Restaurant,Cosmetics Shop,Seafood Restaurant,Art Gallery,American Restaurant,Bar,Bookstore,Market
3,Chelsea,Art Gallery,Coffee Shop,Italian Restaurant,Ice Cream Shop,Hotel,Market,Café,Cycle Studio,Cupcake Shop,Seafood Restaurant
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Optical Shop,Salon / Barbershop,Coffee Shop,Spa,American Restaurant,Asian Restaurant,Hotel


<hr>

## Cluster Neighborhoods

In [34]:
from sklearn.cluster import KMeans

# Number of clusters
k = 5

mnh_group_clustering = mnh_group.drop('Neighborhood', axis=1)
model = KMeans(n_clusters=k, random_state=0).fit(mnh_group_clustering)

model.labels_[:10]

array([1, 1, 1, 0, 0, 1, 1, 3, 0, 1], dtype=int32)

#### Adding the labels to dataframe

In [35]:
neigh_venues_sorted.insert(0, 'Cluster Labels', model.labels_)

mnh_merged = mnh_df
mnh_merged = mnh_merged.join(neigh_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

mnh_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Discount Store,Seafood Restaurant,Tennis Stadium,Supplement Shop,Bank,Steakhouse
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,Optical Shop,Salon / Barbershop,Coffee Shop,Spa,American Restaurant,Asian Restaurant,Hotel
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Pizza Place,Spanish Restaurant,Gym,Coffee Shop,Bank
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Pizza Place,Restaurant,Café,Lounge,Bakery,Pharmacy,Park,Chinese Restaurant,Spanish Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Café,Deli / Bodega,Coffee Shop,Mexican Restaurant,Yoga Studio,Park,Chinese Restaurant,School,Sushi Restaurant


#### Visualizing the clusters

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mnh_merged['Latitude'], mnh_merged['Longitude'], mnh_merged['Neighborhood'], mnh_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.vector_layers.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<hr>

## Examine Clusters

#### Cluster 0

In [37]:
mnh_merged.loc[mnh_merged['Cluster Labels'] == 0, mnh_merged.columns[[1] + list(range(5, mnh_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Optical Shop,Salon / Barbershop,Coffee Shop,Spa,American Restaurant,Asian Restaurant,Hotel
5,Manhattanville,Seafood Restaurant,Coffee Shop,Italian Restaurant,Mexican Restaurant,Fried Chicken Joint,Park,Japanese Curry Restaurant,Cuban Restaurant,Sushi Restaurant,Supermarket
8,Upper East Side,Italian Restaurant,Bakery,Gym / Fitness Center,Juice Bar,Yoga Studio,Hotel,Wine Shop,Exhibit,French Restaurant,Cosmetics Shop
9,Yorkville,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Deli / Bodega,Bar,Wine Shop,Diner,Mexican Restaurant,Japanese Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Cocktail Bar,Café,Gym / Fitness Center,Gym,Burger Joint,Cycle Studio
12,Upper West Side,Italian Restaurant,Bakery,Coffee Shop,Bagel Shop,Vegetarian / Vegan Restaurant,Pizza Place,Sushi Restaurant,Middle Eastern Restaurant,Seafood Restaurant,Bookstore
13,Lincoln Square,Italian Restaurant,Café,Plaza,Concert Hall,Gym / Fitness Center,Performing Arts Venue,Theater,Wine Shop,Indie Movie Theater,American Restaurant
17,Chelsea,Art Gallery,Coffee Shop,Italian Restaurant,Ice Cream Shop,Hotel,Market,Café,Cycle Studio,Cupcake Shop,Seafood Restaurant
18,Greenwich Village,Italian Restaurant,Pizza Place,Coffee Shop,Ice Cream Shop,Wine Bar,Gym,Bakery,Dessert Shop,Restaurant,Park
19,East Village,Pizza Place,Cocktail Bar,Wine Bar,Coffee Shop,Dessert Shop,Vietnamese Restaurant,Japanese Restaurant,Ramen Restaurant,Juice Bar,Gourmet Shop


#### Cluster 1

In [38]:
mnh_merged.loc[mnh_merged['Cluster Labels'] == 1, mnh_merged.columns[[1] + list(range(5, mnh_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Harlem,African Restaurant,Chinese Restaurant,French Restaurant,Cosmetics Shop,Seafood Restaurant,Art Gallery,American Restaurant,Bar,Bookstore,Market
14,Clinton,Theater,Coffee Shop,Gym / Fitness Center,Wine Shop,Hotel,Gym,Sandwich Place,Cocktail Bar,American Restaurant,Spa
15,Midtown,Coffee Shop,Clothing Store,Hotel,Theater,Cuban Restaurant,Pizza Place,Bakery,Spa,Tailor Shop,Mediterranean Restaurant
16,Murray Hill,Sandwich Place,Hotel,Coffee Shop,Steakhouse,Gym / Fitness Center,Pizza Place,Japanese Restaurant,Burger Joint,Grocery Store,Bar
21,Tribeca,Park,Italian Restaurant,Spa,Wine Bar,Café,Men's Store,Steakhouse,Skate Park,Scenic Lookout,Bakery
25,Manhattan Valley,Coffee Shop,Bar,Spa,Pizza Place,Mexican Restaurant,Park,Bubble Tea Shop,Café,Caribbean Restaurant,Chinese Restaurant
28,Battery Park City,Park,Hotel,Gym,Boat or Ferry,Memorial Site,Shopping Mall,Food Court,Beer Garden,Playground,Plaza
29,Financial District,Coffee Shop,Hotel,American Restaurant,Pizza Place,Salad Place,Café,Cocktail Bar,Gym,Gym / Fitness Center,Park
30,Carnegie Hill,Coffee Shop,Yoga Studio,Wine Shop,Pizza Place,Japanese Restaurant,Gym / Fitness Center,Gym,Grocery Store,Café,Bookstore
32,Civic Center,Coffee Shop,Spa,Hotel,French Restaurant,Park,Yoga Studio,American Restaurant,Sushi Restaurant,Gym / Fitness Center,Cocktail Bar


#### Cluster 2

In [39]:
mnh_merged.loc[mnh_merged['Cluster Labels'] == 2, mnh_merged.columns[[1] + list(range(5, mnh_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Boat or Ferry,Park,Bar,Pet Service,Gas Station,German Restaurant,Bistro,Farmers Market,Gym / Fitness Center,Baseball Field


#### Cluster 3

In [40]:
mnh_merged.loc[mnh_merged['Cluster Labels'] == 3, mnh_merged.columns[[1] + list(range(5, mnh_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Pizza Place,Spanish Restaurant,Gym,Coffee Shop,Bank
3,Inwood,Mexican Restaurant,Pizza Place,Restaurant,Café,Lounge,Bakery,Pharmacy,Park,Chinese Restaurant,Spanish Restaurant
4,Hamilton Heights,Pizza Place,Café,Deli / Bodega,Coffee Shop,Mexican Restaurant,Yoga Studio,Park,Chinese Restaurant,School,Sushi Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Deli / Bodega,Mobile Phone Shop,Seafood Restaurant,Taco Place,Street Art,Steakhouse
11,Roosevelt Island,Pizza Place,Scenic Lookout,Coffee Shop,Park,Gym / Fitness Center,Gym,Sandwich Place,Liquor Store,School,Dry Cleaner
26,Morningside Heights,Park,Bookstore,Coffee Shop,American Restaurant,Deli / Bodega,Burger Joint,Tennis Court,College Cafeteria,Café,Mediterranean Restaurant
35,Turtle Bay,Café,Coffee Shop,Wine Bar,Park,Italian Restaurant,Deli / Bodega,Sushi Restaurant,French Restaurant,Plaza,Turkish Restaurant
36,Tudor City,Park,Café,Mexican Restaurant,Pizza Place,Deli / Bodega,Greek Restaurant,Thai Restaurant,Diner,Dog Run,Garden


#### Cluster 4

In [41]:
mnh_merged.loc[mnh_merged['Cluster Labels'] == 4, mnh_merged.columns[[1] + list(range(5, mnh_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Discount Store,Seafood Restaurant,Tennis Stadium,Supplement Shop,Bank,Steakhouse


<hr>